In [31]:
import numpy as np
import random
from math import inf

In [32]:
def get_user_input():
    i,j = map(int,input("enter row and column index from [0,1,2]->"
                        "should be comma separated : ").split(','))
    print(f"i : {i} and j is {j}")
    return i,j

In [33]:
def check_rows_and_columns(playGround,sign):
    
    for i in range(3):
        p = playGround[i,:]
        signWon = all([True if i == sign else False for i in p])
        if signWon:
            return True
        
    for k in range(3):
        q = playGround[:,k]
        signWon = all([True if i == sign else False for i in q])
        if signWon:
            return True         
    
    return False
            
        
# print(check_rows_and_columns(playGround,'-'))        

In [34]:
def goal_test(playGround,sign):
    row_column = check_rows_and_columns(playGround,sign)    

    # first diagnal
    diagnalOne = all([True if playGround[i,i]== sign else False for i in range(3)])
    
    # second diagnal
    diagnalTwo = all([True if playGround[i,2-i]== sign else False for i in range(3)])

    return row_column or diagnalOne or diagnalTwo


In [35]:
def getValidActions(playGround):
    validActions = []
    for i in range(3):
        for j in range(3):
            if playGround[i,j] == '-':
                validActions.append((i,j))
    return validActions


In [36]:
def getLineValue(vector):
    
    val = 0
    
    # first box of the vector
    if vector[0] == 'O':
        val = 1
    elif vector[0] == 'x':
        val = -1
        
    if vector[1] == 'O':
        if val == 1:
            val = 10;
        elif val == -1:
            return 0;
        else:
            val = 1
    elif vector[1] == 'x':
        if val == -1:
            val = -10
        elif val == 1:
            return 0
        else:
            val = -1
        
    
    
    if vector[2] == 'O':
        if val > 0:
            val *= 10
        elif val < 0:
            return 0
        else:
            val = 1
    elif vector[2] == 'x': 
        if val < 0:
            val *= 10
        elif val > 1:
            return 0
        else:
            val = -1
            
            
    return val
            
    

In [37]:
def getAllVectors(playGround):
    diagnalOne = []
    diagnalTwo = []
    vectors = []
    for i in range(3):
        vectors.append(playGround[i,:])
        vectors.append(playGround[:,i])
        
        diagnalOne.append(playGround[i,i])
        diagnalTwo.append(playGround[i,2-i])
        
    vectors.append(diagnalOne)
    vectors.append(diagnalTwo)
    return vectors
        

In [38]:
def getHeauristic(playGround):
    vectors = getAllVectors(playGround)
    p = 0
    
    for vector in vectors:
        p += getLineValue(vector)
    
    return p
    
    

In [39]:
def BasicMinimaxAlgo(playGround,noUserAlpha,noUserBeta,player):
    # if user won
    if goal_test(playGround,'x'):
        return -1,-1,-100
    
    # if computer won
    elif goal_test(playGround,'O'):
        return -1,-1,100
    
    
    availableActions = getValidActions(playGround)
    
    
    if len(availableActions) == 0:
        return -1,-1,0
    
    # neither one won
    data = (-1,-1)
    
    # if player is max
    if player:
        val = -inf
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'O'
            a,b,value = BasicMinimaxAlgo(newplayGround,noUserAlpha,noUserBeta,False)
            if value > val:
                val = value
                data = action
            
    # else if play is min
    else:
        val = inf
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'x'
            a,b,value = BasicMinimaxAlgo(newplayGround,noUserAlpha,noUserBeta,True)
            if value < val :
                val = value
                data = action
    
#     print(data)
            
    return data[0],data[1],val           

In [40]:
def AlphaBetaMinimax(playGround,alpha,beta,player):   
    # if user won
    if goal_test(playGround,'x'):
        return -1,-1,-100
    
    # if computer won
    elif goal_test(playGround,'O'):
        return -1,-1,100
    
    availableActions = getValidActions(playGround)
    
    # neither one won
    if len(availableActions) == 0:
        return -1,-1,0
    
    data = (-1,-1)
    
    # if player is max
    if player:
        val = -inf
        
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'O'
            a,b,value = BasicMinimaxAlgo(newplayGround,alpha,beta,False)
#             print(value)
            if value >= val:
                val = value
                data = action
                
                
            alpha = max(alpha,val)
            if(alpha > beta):
                break
                
            
    # else if play is min
    else:
        val = inf
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'x'
            a,b,value = BasicMinimaxAlgo(newplayGround,alpha,beta,True)
            if value < val :
                val = value
                data = action
            
            beta = min(beta,val)
            if(alpha > beta):
                break
        
            
    return data[0],data[1],val     

In [41]:
def DepthLimitedMinimax(playGround,depth,noUserAlpha,noUserBeta,player):
        # if user won
    if goal_test(playGround,'x'):
        return -1,-1,-100
    
    # if computer won
    elif goal_test(playGround,'O'):
        return -1,-1,100    
    
    availableActions = getValidActions(playGround)
    
    if len(availableActions) == 0:
        return -1,-1,0
    
    if depth == 0:
        return -1,-1,getHeauristic(playGround)
    
    
    
    # neither one won
    data = (-1,-1)
    
    # if player is max
    if player:
        val = -inf
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'O'
            a,b,value = DepthLimitedMinimax(newplayGround,depth,noUserAlpha,noUserBeta,False)
            if value > val:
                val = value
                data = action
            
    # else if play is min
    else:
        val = inf
        depth -= 1
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'x'
            a,b,value = DepthLimitedMinimax(newplayGround,depth,noUserAlpha,noUserBeta,True)
            if value < val :
                val = value
                data = action
    
#     print(data)
            
    return data[0],data[1],val   
    

In [42]:
def DepthLimitAlphaBetaMinimax(playGround,depth,alpha,beta,player):
       # if user won
        
    # update this value as depth will be returning many distinct type of values which could be less than -1
    if goal_test(playGround,'x'):
        return -1,-1,-100
    
    # if computer won
    # update this value as depth --> more than +1
    elif goal_test(playGround,'O'):
        return -1,-1,100    
    
    availableActions = getValidActions(playGround)
    
    # think about this value --> you havent thought about it : instead of 0
    if len(availableActions) == 0:
        return -1,-1,0
    

    if depth == 0:
        return -1,-1,getHeauristic(playGround)
        
    
    availableActions = getValidActions(playGround)
    
    data = (-1,-1)
    
    # if player is max
    if player:
        val = -inf
        
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'O'
            a,b,value = DepthLimitAlphaBetaMinimax(newplayGround,depth,alpha,beta,False)
#             print(value)
            if value >= val:
                val = value
                data = action
                
                
            alpha = max(alpha,val)
            if(alpha > beta):
                break
                
            
    # else if play is min
    else:
        val = inf
        depth -= 1
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'x'
            a,b,value = DepthLimitAlphaBetaMinimax(newplayGround,depth,alpha,beta,True)
            if value < val :
                val = value
                data = action
            
            beta = min(beta,val)
            if(alpha > beta):
                break
        
            
    return data[0],data[1],val     

In [43]:
# def efficientMinimax(newplayGround,depth,alpha,beta,player):
#        # if user won
#     if depth == 0:
#         return -1,-1,getHeauristic(playGround,player)
        
    
#     availableActions = getValidActions(playGround)
    
#     # return the type of symetry
# #     typeofsym,present = getTypeOfSymetry(playground)
    
# #     if present:
#         # remove redundant actions
# #         availableActions = getUpdatedActions(playGround,availableActions)
    
    
#     data = (-1,-1)
    
#     # if player is max
#     if player:
#         val = -inf
        
#         for action in availableActions:
#             newplayGround = playGround.copy()
#             newplayGround[action[0],action[1]] = 'O'
#             a,b,value = efficientMinimax(newplayGround,depth,alpha,beta,False)
# #             print(value)
#             if value > val:
#                 val = value
#                 data = action
                
                
#             alpha = max(alpha,val)
#             if(alpha > beta or val == 1):
#                 break
                
            
#     # else if play is min
#     else:
#         val = inf
#         depth -= 1
        
#         for action in availableActions:
#             newplayGround = playGround.copy()
#             newplayGround[action[0],action[1]] = 'x'
#             a,b,value = efficientMinimax(newplayGround,depth,alpha,beta,True)
#             if value < val :
#                 val = value
#                 data = action
            
#             beta = min(beta,val)
#             if(alpha > beta or val == -1):
#                 break
        
            
#     return data[0],data[1],val   

In [44]:
class Agent:
    
    # initializing with all '-' environment
    def __init__(self,depth):
        self.playGround = np.array([['-','-','-'],['-','-','-'],['-','-','-']])
        self.computerChances = 0
        self.userChances = 0
        self.alpha = -inf
        self.beta = inf
        self.depth = depth
        self.sustain = depth
        
    # if playGround is in other state
    def take_percept(self):
        self.playGround = np.array([['-','-','-'],['-','-','-'],['-','-','-']])
        self.userChances = 0
        self.computerChances = 0
        self.alpha = -inf
        self.beta = inf
        self.depth = self.sustain
        
        

    def setChoiceFunction(self,choice):
        isSet =  True
        function = DepthLimitAlphaBetaMinimax
        
        if choice == 1:
            function = BasicMinimaxAlgo
            isSet = False
        elif choice == 2:
            function = AlphaBetaMinimax
            isSet = False
        elif choice == 3:
            function = DepthLimitedMinimax
            isSet = True
        elif choice == 4 or choice == 5:
            function = DepthLimitAlphaBetaMinimax
            isSet = True
#         else:
#             function = efficientMinimax  # not implemented yet : apply symetry rule , use probability
#             isSet = True
        
        return isSet,function
        
        
        
    # function to run
    # if computer came to know it will loose any way then it will put 'O' at the first place available for it.
    def playGame(self,function,user):
        
        isSet,function = self.setChoiceFunction(choice)
        
        gameOver = False
        while not gameOver:
            print(self.playGround,end='\n\n\n')
            if user:
                i,j = get_user_input()
                self.playGround[i,j]='x'
                self.userChances+=1

                if goal_test(self.playGround,'x'):
                    print('You won')
                    break



            if self.userChances + self.computerChances != 9 :
                user=True
                self.computerChances += 1
                newplayGround = self.playGround.copy()
                if not isSet:
                    i,j,p = function(newplayGround,self.alpha,self.beta,True)
                else:
                    i,j,p = function(newplayGround,self.depth,self.alpha,self.beta,True)
                    

                if i != -1:
                    self.playGround[i,j] = 'O'
                if goal_test(self.playGround,'O'):
                    print('You loose')
                    break
            else:
                print('match draw')
                break

        else:
            print('Match Draw')
            
    
             
             
                
    def showState(self):
        print(self.playGround)
    
    
        
    
    
    

In [45]:
# print('Tic Tock Toe ')
# # chance = input("do you want odd chances or even chances enter e/o : ")

# # choice = int(input("""
# # Choose your algorithm :
# # 1. BasicMinimaxAlgo
# # 2. AlphaBetaMinimax
# # 3. DepthLimitedMinimax
# # 4. DepthLimitAlphaBetaMinimax
# # 5. efficientMinimax
# # """))

# agent = Agent()
# agent.playGame(BasicMinimaxAlgo,True)




print('Tic Toc Toe ')
chance = int(input("do you want odd chances or even chances enter 1 -> eventChances or 2 -> oddChances : "))
player = False if chance==1 else True


agent = Agent(5)

playChoice = True
while playChoice:
    choice = int(input("""
    
    
Choose your algorithm :
1. BasicMinimaxAlgo
2. AlphaBetaMinimax
3. DepthLimitedMinimax
4. DepthLimitAlphaBetaMinimax
5. efficientMinimax


"""))
    
    agent.playGame(player,choice)
    playChoice = False
    
    if int(input('press One to play Again : ')) == 1:
        playChoice = True
        agent.take_percept()

print('Thanks ...')

Tic Toc Toe 
do you want odd chances or even chances enter 1 -> eventChances or 2 -> oddChances : 2

    
    
Choose your algorithm :
1. BasicMinimaxAlgo
2. AlphaBetaMinimax
3. DepthLimitedMinimax
4. DepthLimitAlphaBetaMinimax
5. efficientMinimax


4
[['-' '-' '-']
 ['-' '-' '-']
 ['-' '-' '-']]


enter row and column index from [0,1,2]->should be comma separated : 1,0
i : 1 and j is 0
[['-' '-' '-']
 ['x' '-' '-']
 ['O' '-' '-']]


enter row and column index from [0,1,2]->should be comma separated : 1,1
i : 1 and j is 1
[['-' '-' '-']
 ['x' 'x' 'O']
 ['O' '-' '-']]


enter row and column index from [0,1,2]->should be comma separated : 0,0
i : 0 and j is 0
[['x' '-' '-']
 ['x' 'x' 'O']
 ['O' '-' 'O']]


enter row and column index from [0,1,2]->should be comma separated : 0,2
i : 0 and j is 2
You loose
press One to play Again : 1

    
    
Choose your algorithm :
1. BasicMinimaxAlgo
2. AlphaBetaMinimax
3. DepthLimitedMinimax
4. DepthLimitAlphaBetaMinimax
5. efficientMinimax


2
[['-' 